In [1]:
from transformer_heads import load_headed
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    MistralForCausalLM,
    Trainer,
    BitsAndBytesConfig,
    TrainingArguments,
    GPT2Model,
    GPT2LMHeadModel
)
from transformer_heads.util.helpers import DataCollatorWithPadding
from peft import LoraConfig
from transformer_heads.config import HeadConfig
from transformer_heads.util.model import print_trainable_parameters
from transformer_heads.util.evaluate import evaluate_head_wise,get_top_n_preds
import torch

In [2]:
# model_class, model_path = MistralForCausalLM, "mistralai/Mistral-7B-v0.1", 4096, 32000
model_class, model_path, hidden_size, vocab_size = GPT2LMHeadModel, "gpt2", 768, 50257

In [4]:
heads_configs = [
    HeadConfig(
        name="wikitext_head",
        layer_hook=-4,  # Hook to layer [-4] (Drop 3 layers from the end)
        in_size=hidden_size,
        hidden_size=0,
        num_layers=1,
        output_activation="linear",
        is_causal_lm=True,
        loss_fct="cross_entropy",
        num_outputs=vocab_size,
        is_regression=False,
        output_bias=False,
    )
]

In [5]:
dd = load_dataset("wikitext", "wikitext-2-v1")

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    out = tokenizer(examples["text"], padding="max_length", truncation=True)
    out[heads_configs[0].name] = out["input_ids"].copy()
    return out


for split in dd.keys():
    dd[split] = dd[split].map(tokenize_function, batched=True)
dd.set_format(
    type="torch", columns=["input_ids", "attention_mask", heads_configs[0].name]
)
for split in dd.keys():
    dd[split] = dd[split].remove_columns("text")

In [7]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = load_headed(
    model_class,
    model_path,
    head_configs=heads_configs,
    quantization_config=quantization_config,
    device_map={"": torch.cuda.current_device()},
)

Some weights of TransformerWithHeads were not initialized from the model checkpoint at gpt2 and are newly initialized: ['heads.wikitext_head.lins.0.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
print_trainable_parameters(model)

all params: 120569856 || trainable params: 38597376 || trainable%: 32.01245923359152
params by dtype: defaultdict(<class 'int'>, {torch.float32: 78102528, torch.uint8: 42467328})
trainable params by dtype: defaultdict(<class 'int'>, {torch.float32: 38597376})


In [9]:
dd["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'wikitext_head'],
    num_rows: 36718
})

In [10]:
print(get_top_n_preds(5,model,"The historical significance of",tokenizer,"wikitext_head"))

['Ian', ' bedroom', 'til', ' irregular', 'Cooldown']


In [11]:
args = TrainingArguments(
    output_dir="linear_probe_test",
    learning_rate=0.0002,
    num_train_epochs=0.05,  # To speed things up set to 0.1, set to 1 for better performance
    logging_steps=20,
    do_eval=False,
    remove_unused_columns=False,
)
collator = DataCollatorWithPadding(
    feature_name_to_padding_value={
        "input_ids": tokenizer.pad_token_id,
        heads_configs[0].name: -100,
        "attention_mask": 0,
    }
)
trainer = Trainer(
    model,
    args=args,
    train_dataset=dd["train"],
    data_collator=collator,
)
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ykeller (chm-hci). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
20,14.700400
40,5.302300
60,1.792900
80,0.884400
100,0.772400
120,0.706800
140,0.687500
160,0.717300
180,0.562600
200,0.615500


TrainOutput(global_step=230, training_loss=2.3977216720581054, metrics={'train_runtime': 149.8647, 'train_samples_per_second': 12.25, 'train_steps_per_second': 1.535, 'total_flos': 1397896469544960.0, 'train_loss': 2.3977216720581054, 'epoch': 0.05})

In [12]:
print(evaluate_head_wise(model, dd["validation"], collator, epochs=0.1))

Evaluating:   0%|          | 0/470 [00:00<?, ?it/s]/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Evaluating:  10%|█         | 47/470 [00:31<04:45,  1.48it/s]

(0.5419302958374222, {'wikitext_head': 0.5419302958374222})


In [13]:
print(get_top_n_preds(5,model,"The historical significance of",tokenizer,"wikitext_head"))

[' the', ' <', ' it', ' his', ' her']
